In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fake.csv')

In [5]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [0]:
df=pd.get_dummies(df,columns= ['label'],drop_first=True)

In [0]:
x=df.drop('label_REAL',1)
y=df['label_REAL']

In [11]:
y.value_counts()

1    3171
0    3164
Name: label_REAL, dtype: int64

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot
import nltk
import re
from nltk.corpus import stopwords
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from nltk.stem.porter import PorterStemmer

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
v_size=10000

In [0]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(x)):
    review = re.sub('[^a-zA-Z]', ' ', x['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [0]:
onehot=[one_hot(words,v_size)for words in corpus] 

In [20]:
onehot

[[7899, 7539, 8797],
 [2711, 5191, 4814, 2266, 1734, 6986, 4959, 5267, 2243, 3722, 5110],
 [8333, 2367, 8003, 2337, 3279],
 [2767, 3346, 2451, 3364, 9797, 1238, 6834, 2537],
 [3668, 7848, 7892, 5230, 5276],
 [7506, 6985],
 [9038, 7192, 2711, 8438, 7385, 3457],
 [7896, 5458, 8098],
 [3607, 7801, 2243, 8762, 2230, 613, 7304],
 [1292, 9122, 5916, 7848, 6653, 9689, 8569, 9982, 3569],
 [8728, 8762, 7174, 6696, 2543, 7870, 7539, 8762, 1011],
 [2565, 2243, 6206, 8747, 5816, 2506, 667, 8820, 5591, 4606],
 [650, 9934, 4110, 4273, 737, 3255, 3041, 4206, 5110],
 [9636, 9562, 5617, 1441, 9307],
 [2243, 5449, 5772, 9658],
 [8109, 9173, 1548],
 [8756, 5698, 3073, 7539, 3677, 38, 7777, 8811, 7891],
 [7539,
  8762,
  8215,
  2383,
  9562,
  8016,
  2612,
  4894,
  5829,
  1605,
  5600,
  9200,
  3041],
 [1292, 3672, 3073, 733],
 [1447, 4494, 8979, 9866, 7358, 7129, 9562],
 [2993, 1863, 2243, 8614, 8105, 5912],
 [8387, 9741, 5110, 297, 6253, 4646, 278, 8268, 5464],
 [7539, 8762, 5916, 8033, 7390, 6493,

In [23]:
length=20
embedded=pad_sequences(onehot,padding='pre',maxlen=length)
embedded

array([[   0,    0,    0, ..., 7899, 7539, 8797],
       [   0,    0,    0, ..., 2243, 3722, 5110],
       [   0,    0,    0, ..., 8003, 2337, 3279],
       ...,
       [   0,    0,    0, ..., 4532, 2605, 2387],
       [   0,    0,    0, ..., 8494, 4674, 3783],
       [   0,    0,    0, ..., 4051, 2243, 5276]], dtype=int32)

In [35]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(v_size,embedding_vector_features,input_length=length))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 100)               36400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 436,501
Trainable params: 436,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
X=np.array(embedded)
Y=np.array(y)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [39]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0135 - accuracy: 0.9960 - val_loss: 1.4681 - val_accuracy: 0.7580
Epoch 2/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0054 - accuracy: 0.9986 - val_loss: 1.5959 - val_accuracy: 0.7547
Epoch 3/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 1.5750 - val_accuracy: 0.7585
Epoch 4/100
67/67 [==============================] - 1s 12ms/step - loss: 0.0031 - accuracy: 0.9991 - val_loss: 1.6803 - val_accuracy: 0.7575
Epoch 5/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 1.7369 - val_accuracy: 0.7556
Epoch 6/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 1.7846 - val_accuracy: 0.7575
Epoch 7/100
67/67 [==============================] - 1s 11ms/step - loss: 0.0031 - accuracy: 0.9988 - val_loss: 1.7899 - val_accuracy: 0.7532
Epoch 

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred=model.predict_classes(x_test)
confusion_matrix(y_test,y_pred)

array([[816, 255],
       [251, 769]])

In [41]:
accuracy_score(y_test,y_pred)

0.7580105212816834